In [ ]:
import os, re
from IPython.display import display
from arcgis.gis import GIS
import pandas as pd


''' ********************** SCRIPT CONFIGURATION START ********************** '''

#What is the ID of the Feature Layer you want to Add attachments to?
FeatureLayerId = ''

#What are your ArcGIS Enterprise/ArcGIS Online credentials? This is case sensitive.
PortalUserName = ''
PortalPassword = ''
PortalUrl = ''

#Where are your attachments stored?
attachmentFolder = r""
'''******************************END CONFIGURATION****************************'''
attachmentCounter = 0
existsCounter = 0
#Set to False if ArcGIS Enterprise cert is not valid
PortalCertVerification = True
#Connect to GIS, and get Feature Layer information
if PortalUserName == '' and PortalPassword == '':
    gis = GIS()
else:
    gis = GIS(PortalUrl, PortalUserName, PortalPassword, verify_cert=PortalCertVerification)
#Display item
itemObject = gis.content.get(FeatureLayerId)
display(itemObject)

#Loop through layers in Feature Layer
for i in range(len(itemObject.layers)):
    featureLayer = itemObject.layers[i]
    #Skip layer if attachments are not enabled
    if featureLayer.properties.hasAttachments == True:
        #Query to get list of all records to start with
        query1 = featureLayer.query(where='1=1', return_all_records='true')
        #Create dataframe object to get list of attributes to match attachments to
        df = query1.df
        df.sort_index()
    #Iterate through dataframe to get ObjectID's and GlobalID's 
    for row in df.itertuples():
        ObjectID = row.OBJECTID
        globalid = row.GlobalID
        #Get list of attachments for each ObjectID
        featureAttachments = featureLayer.attachments.get_list(oid=ObjectID)
        #If ObjectID has no attachments continue to adding attachments
        if len(featureAttachments) == 0:
            #Look in specified folder for files to attach
            for root, dirs, files in os.walk(attachmentFolder): 
                #List files in folder
                for filename in files:
                    #find all files that have the global id of a feature in their name
                    attachment = re.findall((globalid), filename, re.IGNORECASE)
                    if not attachment: continue               
                    for pic in attachment:                   
                        #find individual feature globalid's that match those contained in files 
                        features = df.GlobalID.str.upper() == pic
                        #use globalID to isolate feature Object Id's
                        for row in df[features].itertuples():
                            realOID = row.OBJECTID
                            filePath = os.path.abspath(os.path.join(root, filename))
                            #attach the appropriate files
                            featureLayer.attachments.add(realOID, filePath)
                            attachmentCounter += 1
                            print('Added Attachment: {} to ObjectID {}'.format(filename, realOID))
                            print('{} attachments added so far...'.format(attachmentCounter))
        #if features already have attachments                     
        else:
            attachments = [a for a in featureLayer.attachments.get_list(oid=ObjectID)]            
            df2 = pd.DataFrame(attachments)            
            attachList = []
            for row in df2.itertuples():
                attId = row.globalId
                parentId = row.parentGlobalId
                name = row.name                
                updated = re.findall((parentId), name, re.IGNORECASE)                
                if updated:
                    attachList.append(name)                   
                else:
                    attachList.append('{' + parentId.upper() +'}' + '_' + '{' + attId.upper() + '}' + '_' + name) 
                continue          
            for root, dirs, files in os.walk(attachmentFolder):             
                for filename in files:

                    if filename in attachList:
                        existsCounter += 1
                        print('Attachment:{} already exists.'.format(filename))
                    else:
                        attachment = re.findall((globalid), filename, re.IGNORECASE)
                        if not attachment: continue               
                        for pic in attachment:
                            features = df.GlobalID.str.upper() == pic
                            for row in df[features].itertuples():
                                realOID = row.OBJECTID
                                filePath = os.path.abspath(os.path.join(root, filename))
                                #attach the appropriate files
                                featureLayer.attachments.add(realOID, filePath)
                                attachmentCounter += 1
                                print('Added Attachment: {} to ObjectID {}'.format(filename, realOID))
                                print('{} attachments added so far...'.format(attachmentCounter))
print('***************************************************************************************************************')
print('{} files already exist as attachments'.format(existsCounter))
print('{} newly created attachments'.format(attachmentCounter))